In [2]:
ler_cota_hist <- function(data){
  stopifnot(is(data, "Date"), length(data) == 1)
  url <- format(data,
                "http://bvmf.bmfbovespa.com.br/InstDados/SerHist/COTAHIST_D%d%m%Y.ZIP")
  filename <- format(data, "Gabriel André/Estudo R/COTAHIST_D%d%m%Y.ZIP")
  download.file(url = url, destfile = filename, mode = 'wb')
  file <- unzip(filename, exdir = "Gabriel André/Estudo R")
  file.remove(filename)
  layout <- read.csv2("Gabriel André/Estudo R/Excel/layout_cotahist.csv",
                      stringsAsFactors = F)
  dados <- read.fwf(file = file, widths = layout$tamanho, header = F,
                    col.names = layout$campo, skip = 1,
                    stringsAsFactors = F)
  file.remove(file)
  dados <- dados[-nrow(dados),]
  dados$DATA_PREGAO <- as.Date(dados$DATA_PREGAO, "%Y%m%d")
  dados$DATVEN <- as.Date(as.character(dados$DATVEN), "%Y%m%d")
  cols <- c("PREABE", "PREMAX", "PREMIN", "PREMED", "PREULT", 
            "PREOFC", "PREOFV", "PREEXE")
  for(nome_coluna in cols){
    dados[,nome_coluna] <- dados[,nome_coluna]/100
  }
  dados["Variação diária"] <- round((dados$PREULT-dados$PREABE)/dados$PREABE*100, 3)
  dados$VOLTOT <- dados$VOLTOT/100
  dados["Variação máxima"] <- round((dados$PREMAX-dados$PREMIN)/dados$PREMIN*100,3)
  return(View(dados))
}

In [1]:
grafico_cota_hist <- function(data, top_n = NULL){
  stopifnot(is(data, "Date"), length(data) == 1, length(top_n) == 1)
  pacotes <- c("ggplot2", "dplyr", "magrittr", "ggplot2", "ggthemes")
  instalados <- installed.packages()
  faltam <- pacotes[!(pacotes %in% instalados[,1])]
  if(length(faltam) > 0){
    install.packages(faltam)
    for(i in 1:length(faltam)){
      message(paste(faltam[i], "instalado"))
    }
  } else {
    if(length(faltam) == 0){
      for(i in pacotes){
        library(i, character.only = T)
      }
    }
  }
  url <- format(data,
                "http://bvmf.bmfbovespa.com.br/InstDados/SerHist/COTAHIST_D%d%m%Y.ZIP")
  filename <- format(data, "Gabriel André/Estudo R/COTAHIST_D%d%m%Y.ZIP")
  download.file(url = url, destfile = filename, mode = 'wb')
  file <- unzip(filename, exdir = "Gabriel André/Estudo R")
  file.remove(filename)
  layout <- read.csv2("Gabriel André/Estudo R/Excel/layout_cotahist.csv",
                      stringsAsFactors = F)
  dados <- read.fwf(file = file, widths = layout$tamanho, header = F,
                    col.names = layout$campo, skip = 1,
                    stringsAsFactors = F)
  file.remove(file)
  dados <- dados[-nrow(dados),]
  dados$DATA_PREGAO <- as.Date(dados$DATA_PREGAO, "%Y%m%d")
  dados$DATVEN <- as.Date(as.character(dados$DATVEN), "%Y%m%d")
  cols <- c("PREABE", "PREMAX", "PREMIN", "PREMED", "PREULT", 
            "PREOFC", "PREOFV", "PREEXE")
  for(nome_coluna in cols){
    dados[,nome_coluna] <- dados[,nome_coluna]/100
  }
  dados["Variação diária"] <- round((dados$PREULT-dados$PREABE)/dados$PREABE*100, 3)
  dados$VOLTOT <- dados$VOLTOT/100
  dados["Variação máxima"] <- round((dados$PREMAX-dados$PREMIN)/dados$PREMIN*100,3)
  if(is.null(top_n)){
    return(dados %>%
             ungroup() %>%
             top_n(10, VOLTOT) %>%
             mutate(CODNEG = reorder(CODNEG, VOLTOT)) %>%
             ggplot()+
             geom_col(aes(x = VOLTOT, y = CODNEG))+
             labs(x = "Volume total R$", y = "", title = "10 ativos com maior volume",
                  subtitle = format(data,"%d/%m/%Y"), caption = "Fonte:B3")+
             theme_economist())
  } else{
    if(!is.null(top_n)){
      return(dados %>%
               ungroup() %>%
               top_n(top_n, VOLTOT) %>%
               mutate(CODNEG = reorder(CODNEG, VOLTOT)) %>%
               ggplot()+
               geom_col(aes(x = VOLTOT, y = CODNEG))+
               labs(x = "Volume total R$", y = "", title = if(top_n > 0){
                 paste(abs(top_n), "ativos com maior volume")
               }else{
                 paste(abs(top_n), "ativos com menor volume")
               },
               subtitle = format(data,"%d/%m/%Y"), caption = "Fonte:B3")+
               theme_economist())
    }
  }
}